In [1]:
from dotenv import load_dotenv
load_dotenv() 

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-04-17")

### String output parsers

In [2]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [3]:
from langchain_core.prompts import PromptTemplate

prompt_template_1 = PromptTemplate(template="provide a detailed report on {topic}",
                                   input_variables=['topic'])

prompt_template_2 = PromptTemplate(template="""give a summary on the below topic in 2 sentence. 
                                   {texts}""",
                                   input_variables=['texts'])

In [4]:
chain = prompt_template_1 | llm | parser | prompt_template_2 | llm | parser

In [5]:
chain.invoke({"topic":"Vibe Coding"})

'Vibe Coding is a modern, synchronous, collaborative software development practice for remote teams, evolving from pair programming using advanced shared coding tools. It focuses on creating a shared state of flow and high-bandwidth interaction ("the vibe") to enhance code quality, accelerate knowledge transfer, and improve team cohesion.'

### json output parsers

In [6]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()

In [7]:
template = PromptTemplate(
    template="""
    give me recent statistics of male vs female population min India. 
    {format_instructions}
    """,
    input_variables=[],
    partial_variables={'format_instructions': parser.get_format_instructions()}
)

In [8]:
chain = template | llm | parser

chain.invoke({})

{'country': 'India',
 'reference_period': 'Estimates for July 1, 2023',
 'source': 'United Nations, World Population Prospects 2022 Revision',
 'total_population': 1428627663,
 'male_population': 742416708,
 'female_population': 686210955,
 'sex_ratio_females_per_1000_males': 924.3,
 'notes': 'These figures are estimates/projections based on demographic models. Different sources or survey methodologies (like NFHS-5, 2019-21, which reported a sex ratio of 1020 females per 1000 males for the population surveyed) may yield slightly different results.'}

in jsonOutputParser we can't enforce the structure of the output provided by llms

### structured output parsers

In [9]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

schema = [
    ResponseSchema(name='fact_1', description='Fact 1 about the topic'),
    ResponseSchema(name='fact_2', description='Fact 2 about the topic'),
    ResponseSchema(name='fact_3', description='Fact 3 about the topic'),
]

parser = StructuredOutputParser.from_response_schemas(schema)

template = PromptTemplate(
    template='Give 3 fact about {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

chain = template | llm | parser

chain.invoke({'topic':'black hole'})

{'fact_1': 'A black hole is a region of spacetime where gravity is so strong that nothing, including light and other electromagnetic waves, has enough energy to escape its event horizon.',
 'fact_2': 'Most black holes are formed from the gravitational collapse of a massive star at the end of its life.',
 'fact_3': 'The boundary around a black hole beyond which nothing can escape is called the event horizon.'}

Disadvantages:
- can't validate dtypes

### Pydantic output parsers

In [10]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Optional, Literal

class Review(BaseModel):
    
    name: Optional[str] = Field(None, description="Name of the person")
    product: str = Field(..., description="Product name")
    sentiment: Literal['Positive', "Neutral", "Negative"] = Field(..., description="sentiment of the comment")
    
parser = PydanticOutputParser(pydantic_object = Review)

template = PromptTemplate(
    template="""{review}
    {structured_instruction}""",
    input_variables=["review"],
    partial_variables= {"structured_instruction" :parser.get_format_instructions()}
)

In [11]:
chain = template | llm | parser

chain.invoke(
    {"review":"""
	ANJANA JYOTHY
1.0 out of 5 stars Worst brand and worst service
Reviewed in India on 11 March 2024
Style Name: 2024 - 1.5 Ton 3 StarVerified Purchase
Bought it for my Kerala home and thought Daikin is a know brand and I would like it. But, I was wrong. From day 1, the cooling of the AC is abominable. It starts slowly and reaches at around 90-97% compressor capacity and after a little time starts reducing its capacity and stays at 45-55% then after. Reported the issue to the company and they sent technician from some Giant Air System and they say that ambient temperature is high and that’s why it’s like that. I wonder that if 34 degrees ambient temperature is high for this AC, what would happen to people who buy it for Delhi or places like that? It’s been more than 25 days since I’ve registered the complaint and have called Giant Air Systems multiple times but till date they have not attended it.
My suggestion would is people, don’t buy this brand. Their service is poor and you will regret your decision like I am right now.

Update 14/03/24 - The guy (I think he is some senior tech guy from service team) visited on 13/02/24 and checked the system with gas pressure monitor and found nothing wrong with the gas pressure. It was around 140 when operating and he told me that it’s fine. Then apparently he flushed the gas a little bit and told me that sometimes it’s needed to do so to let the trapped air release from the system and it can help. He did check rest of the operations. He explained a lot of things and some key takeaways are-
1. The display room temperature is always adjusted 2-3 degrees higher than the actual room temperature so that the machine could keep on working on keeping the room temperature to a comfortable levels when people open the doors or windows of that room for something.
2. He taught me how without any equipment one can test and find that the AC gas pressure and other functions are ok. The method is - just go to the outdoor unit and check the pipe temperature of both in and out pipes and if they are somewhat similar in temperature, that means the rotation of the gas is perfect and there is no need to worry. On the other hand if there is a big variation in temperature of in and out pipes, one should ask the technician to come and check.
3. Even if we set 18 degree as the required temperature for the room, it will never really reach that level in practical conditions. So, we must be ok with that as well and I think that’s fair enough.
4. He asked me to keep the AC at 24 degrees preferably and that would give you the most comfortable room condition and I agree with him.
5. These are smart ACs and they are designed to operate with minimal power consumption and hence you may find the load on compressor varying at times.

He said sometimes the trapped air in the system can cause a bit issue with cooling but I am not sure how and if that air can be removed from the system when it is already mixed with the gas in compressor.

First day of use after flushing it was better compared to previous days. The system didn’t drop performance and did some work to cool the room. At 24 degrees, it felt cool enough and I think it’s improved.
Will share more after a few days of checking it. Increasing the rating from 1 star to 3 star for the well behaved technician from Giant air systems ( actually every technician who visited from Giant was well behaved and gentleman, it’s just my frustration with the delay in their response which got be cribbing about them ;) ha ha haa…)
     """}
)

Review(name='ANJANA JYOTHY', product='Daikin AC', sentiment='Neutral')

In [12]:
template.invoke({'review':"good"})

StringPromptValue(text='good\n    The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"anyOf": [{"type": "string"}, {"type": "null"}], "default": null, "description": "Name of the person", "title": "Name"}, "product": {"description": "Product name", "title": "Product", "type": "string"}, "sentiment": {"description": "sentiment of the comment", "enum": ["Positive", "Neutral", "Negative"], "title": "Sentiment", "type": "string"}}, "required": ["product", "sentiment"]}\n```')

In [13]:
chain.get_graph().print_ascii()

      +-------------+      
      | PromptInput |      
      +-------------+      
             *             
             *             
             *             
    +----------------+     
    | PromptTemplate |     
    +----------------+     
             *             
             *             
             *             
+------------------------+ 
| ChatGoogleGenerativeAI | 
+------------------------+ 
             *             
             *             
             *             
 +----------------------+  
 | PydanticOutputParser |  
 +----------------------+  
             *             
             *             
             *             
        +--------+         
        | Review |         
        +--------+         
